In [1]:
##################################################  IMPORTS
import pandas as pd
import pprint
import os
import numpy as np
import time
import math

import sys
#from tabulate import tabulate

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [2]:
##################################################  AUX FUNCTIONS:
from operator import itemgetter as it
#Does the same as unpivot_from_resultdf_to_dict, get_topn and pivot_back_with_topn TOGHETER
#but Im keeping the functions above in case I need to include more logic that I cant on this one
def get_top5_on_lis_of_labels(dfResult, submission):
    lis_of_labels = []
    for _, row in dfResult.iterrows():
        if submission:
            for option in (map(it(0), row[1:].order(ascending=0)[:5].iteritems())):
                lis_of_labels.append([row['id'], option])
        else:        
            lis_of_labels.append(map(it(0), row[1:].order(ascending=0)[:5].iteritems()))
    return lis_of_labels


def get_top5_on_lis_of_labels2F(dfResult, submission):
    list_of_labels = []    

    for _, row in dfResult.iterrows():
        l = []
        done = 0
        pred_country = 'error'
        for option in (
                        map (
                            it(0,1), row[1:].order(ascending=0)[:5].iteritems()
                            )
                      ):
            country = option[0]
            prob = option[1]
            if done == 0:
                pred_country = country
                #if prob >= 0.9: 
                #    pred_country = country
                #    done = 1
                #if prob <= 0.3:                 
                #    pred_country = 'NDF'
                #    done = 1
            if submission:
                list_of_labels.append([row['id'], pred_country]) #for submissions
            else:
                l.append(pred_country)   #for train CV
        
        if submission == False:
            list_of_labels.append(l)
            
    return list_of_labels    


ndfTH = 0.2
def get_top5_on_lis_of_labels2(dfResult, submission):
    list_of_labels = []    

    for _, row in dfResult.iterrows():
        l = []
        done = 0
        pred_country = 'error'
        i = 5
        
        
        if max(row[1:11]) <= 0.35:
            i = 4
            if submission:
                list_of_labels.append([row['id'], 'NDF'])
            else:
                l.append('NDF') 

        
#         if row['AU'] < ndfTH and row['CA'] < ndfTH and row['DE'] < ndfTH and row['ES'] <ndfTH and row['FR'] < ndfTH and row['GB'] < ndfTH and row ['IT'] < ndfTH and row['NL'] < ndfTH and row['PT'] < ndfTH:
#             if row['US'] < 0.5:
#                 if row['other'] < 0.3:
#                     pred_country = 'NDF'
#                     done = 1
        
        
#         if row['FR'] > 0.20:
#             i = 4
#             if submission:
#                 list_of_labels.append([row['id'], 'FR'])
#             else:
#                 l.append('FR') 
            
        for option in (
                        map (
                            it(0,1), row[1:].order(ascending=0)[:i].iteritems()
                            )
                      ):
            country = option[0]
            prob = option[1]
            if done == 0:
                pred_country = country
                #if prob > 0.9 and country == 'US':
                #    pred_country = country
                #    done = 1                    
                #if prob >= 0.8: 
                #    pred_country = country
                #    done = 1
                #if prob <= 0.3:                 
                #    pred_country = 'NDF'
                #    done = 1
            if submission:
                list_of_labels.append([row['id'], pred_country]) #for submissions
            else:
                l.append(pred_country)   #for train CV
        
        if submission == False:
            list_of_labels.append(l)
            
    return list_of_labels  


#Unpivot from a dataframe where each prediction is a column to a dict where each prediciton is a row:
def unpivot_from_resultdf_to_dict(dfResult):
    result = {}
    for key in dfResult:
        if key != 'id':
            for i in xrange(0,len(dfResult['id'])):
                if dfResult['id'][i] not in result:
                    result[dfResult['id'][i]] = []
                result[dfResult['id'][i]] += [(key,dfResult[key][i])]
    return result


def get_topn(result,n):
    #Use this to get the top value for each option
    return [x[0] for x in sorted(result,key=lambda x:-x[1])[0:min(len(result),n)]]


def pivot_back_with_topn(result,n, submission,dfResult):
    #keys = sorted([k for k in result])
    #keys = ([k for k in result])
    auxindex = []
    for i in dfResult['id']:
        auxindex.append(i)
    
    keys = ([k for k in auxindex])
    list_of_labels = []
    for key in keys:
        if submission:
            for option in get_topn(result[key],n):
                #print str(key) + ',' + str(option)
                list_of_labels.append([str(key), str(option)] )
        else:
            list_of_labels.append(get_topn(result[key],n))
    return list_of_labels 




#SCORE 
def dcg_at_k(r, k, method=1):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k=5, method=1):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max


def score_predictions(preds, truth, n_modes=5):
    """
    preds: pd.DataFrame
      one row for each observation, one column for each prediction.
      Columns are sorted from left to right descending in order of likelihood.
    truth: pd.Series
      one row for each obeservation.
    """
    assert(len(preds)==len(truth))
    r = pd.DataFrame(0, index=preds.index, columns=preds.columns, dtype=np.float64)
    for col in preds.columns:
        r[col] = (preds[col] == truth) * 1.0
    
    score = pd.Series(r.apply(ndcg_at_k, axis=1, reduce=True), name='score')
    return score

In [3]:
##################################################  FILE LOAD AND PRE-PROCESS
#my_train_plus_session3  contains data for action type and action_detail - only rows with sessions - based on pivot_session.csv
#                        scores  0.854761 \ 0.853916 local and  0.87953 LB      73815 train rows
#my_train_plus_session4  same as 3 but conains rows without sessions            213451 train rows  
file = ['my_train_plus_session10.csv', 'age_gender_bkts.csv', 'countries.csv',  'my_test_plus_session10.csv'] #'sessions.csv',
data = {}
for f in file:
    data[f.replace('.csv','')]=pd.read_csv('C:\\git\\Airbnb\\data\\'+f)

#Get train and test data##########################################################################
train = data['my_train_plus_session10']
test = data['my_test_plus_session10']
train = train.fillna(-1)
test = test.fillna(-1)


#train = train[train['country_destination'] != 'NDF']
#train = train[train['country_destination'] != 'US']
#there are 6065 rows on the train data with first_affiliate_tracked null and only 20 on the test data
#train = train[train['first_affiliate_tracked'] != -1]


#Remove ids and prediction field and keep them separate##########################################
train_ids = train['id']
train = train.drop(['id'],axis=1)

target = train['country_destination']
#train = train.drop(['country_destination'],axis=1)
test['country_destination'] = '-'

test_ids = test['id']
test = test.drop(['id'],axis=1)
#np.unique(train[['age']].values)


#This is the Model2 Evaluation target data frame
dfpivottarget = pd.DataFrame(target)
dfpivottarget['AU'] = (dfpivottarget['country_destination'] =='AU')
dfpivottarget['CA'] = (dfpivottarget['country_destination'] =='CA')
dfpivottarget['DE'] = (dfpivottarget['country_destination'] =='DE')
dfpivottarget['ES'] = (dfpivottarget['country_destination'] =='ES')
dfpivottarget['FR'] = (dfpivottarget['country_destination'] =='FR')
dfpivottarget['GB'] = (dfpivottarget['country_destination'] =='GB')
dfpivottarget['IT'] = (dfpivottarget['country_destination'] =='IT')
dfpivottarget['NDF'] = (dfpivottarget['country_destination'] =='NDF') #initially commented for M2, but needed for 2F
dfpivottarget['NL'] = (dfpivottarget['country_destination'] =='NL')
dfpivottarget['other'] = (dfpivottarget['country_destination'] =='other')
dfpivottarget['PT'] = (dfpivottarget['country_destination'] =='PT')
dfpivottarget['US'] = (dfpivottarget['country_destination'] =='US')

In [4]:
len(train.columns)

494

In [5]:
######################       APPLY LOGIC ON TRAIN AND TESTS 
len_train_before = len(train)
len_test_before = len(test)

trainlen = train.shape[0]
dfall = pd.concat((train, test), axis=0, ignore_index=True)

In [6]:
dfall = dfall.drop(['fa_year','ac_year','ac_day','ac_month','fa_day','fa_month'],axis=1) 
dfall = dfall.drop(['-unknown-_wishlists'],axis=1) 
#dfall = dfall.drop(['accept_decline'],axis=1) #all values are zeros on train, only one ID on test
#dfall = dfall.drop(['acculynk_bin_check_success','acculynk_load_pin_pad','acculynk_pin_pad_inactive','acculynk_session_obtained'], axis=1)

In [7]:
#set whoever did more than 1 bookng requestes to 1
#dfall.loc[dfall.num_AT_booking_request >=1, 'num_AT_booking_request'] = 1
#dfall.loc[dfall.guest_booked_elsewhere >=1, 'guest_booked_elsewhere'] = 1

In [8]:
# keep = ['country_destination', 'gender','signup_method','signup_flow','language','affiliate_channel','affiliate_provider','first_affiliate_tracked',
#         'signup_app','first_device_type','first_browser','num_AT_booking_request','num_AD_pending','num_actions',
#         'start_from_zero','device_from_sessions','bad_devicetype_captured','age',
#         'confirm_email_confirm_email', 'confirm_email_link_confirm_email', 'request_new_confirm_email_request_new_confirm_email',
#         'view_search_results_ajax_get_results', 'view_search_results_search', 'view_search_results_search_results', 
#         'view_search_results_index']

# dfall = dfall.ix[:,keep]

In [9]:
dfall['received_confirm_email'] = dfall['confirm_email_confirm_email'] + dfall['confirm_email_link_confirm_email'] + dfall['request_new_confirm_email_request_new_confirm_email']
dfall = dfall.drop(['confirm_email_confirm_email', 'confirm_email_link_confirm_email', 'request_new_confirm_email_request_new_confirm_email' ],axis=1) 

In [10]:
dfall.loc[dfall.received_confirm_email >=1, 'received_confirm_email'] = 1

In [11]:
dfall['view_search_resultsTotal'] = dfall['view_search_results_ajax_get_results'] + dfall['view_search_results_search'] + dfall['view_search_results_search_results'] + dfall['view_search_results_index']
dfall = dfall.drop(['view_search_results_ajax_get_results', 'view_search_results_search', 'view_search_results_search_results', 'view_search_results_index'],axis=1) 

In [12]:
#AGE MANIPULATION
#rememeber that I set everything <14 and >100 as -1 of the way in
dfall.loc[(dfall.age <=14), 'age_bucket'] = '-1' # 0-4 / 5-9 / 10-14
dfall.loc[(dfall.age >=15) & (dfall.age <= 19), 'age_bucket'] = '15-19'
dfall.loc[(dfall.age >=20) & (dfall.age <= 24), 'age_bucket'] = '20-24'
dfall.loc[(dfall.age >=25) & (dfall.age <= 29), 'age_bucket'] = '25-29'
dfall.loc[(dfall.age >=30) & (dfall.age <= 34), 'age_bucket'] = '30-34'
dfall.loc[(dfall.age >=35) & (dfall.age <= 39), 'age_bucket'] = '35-39'
dfall.loc[(dfall.age >=40) & (dfall.age <= 44), 'age_bucket'] = '40-44'
dfall.loc[(dfall.age >=45) & (dfall.age <= 49), 'age_bucket'] = '45-49'
dfall.loc[(dfall.age >=50) & (dfall.age <= 54), 'age_bucket'] = '50-54'
dfall.loc[(dfall.age >=55) & (dfall.age <= 59), 'age_bucket'] = '55-59'
dfall.loc[(dfall.age >=60) & (dfall.age <= 64), 'age_bucket'] = '60-64'
dfall.loc[(dfall.age >=65) & (dfall.age <= 69), 'age_bucket'] = '65-69'
dfall.loc[(dfall.age >=70) & (dfall.age <= 74), 'age_bucket'] = '70-74'
dfall.loc[(dfall.age >=75) & (dfall.age <= 79), 'age_bucket'] = '75-79'
dfall.loc[(dfall.age >=80) & (dfall.age <= 84), 'age_bucket'] = '80-84'
dfall.loc[(dfall.age >=85) & (dfall.age <= 89), 'age_bucket'] = '85-89'
dfall.loc[(dfall.age >=90) & (dfall.age <= 94), 'age_bucket'] = '90-94'
dfall.loc[(dfall.age >=95) & (dfall.age <= 99), 'age_bucket'] = '95-99'
dfall.loc[(dfall.age >=100), 'age_bucket'] = '100+'
dfall = dfall.drop(['age'],axis=1)

In [13]:
#Population in K by age bucket based on users age bucket
popInKByAgeBucket=pd.read_csv('C:\\git\\Airbnb\\aux_data\\popInKByAgeBucket.csv')
dfall = pd.merge(dfall, popInKByAgeBucket, on='age_bucket', how='left').fillna(0)

In [14]:
######################       Hot Encode THE DATASET's COLUMNS - apply one hot encoding
#need to be on the joined dataset to be sure they end up with the same colums
extra_one_hot = ['signup_flow',  'num_AT_booking_request', 'num_AD_pending', 'ajax_google_translate_reviews', 'received_confirm_email']
                 #'start_from_zero', 'accept_decline', 'guest_booked_elsewhere']


for col in dfall.columns:
    if (col != 'country_destination'):
        if ( (dfall[col].dtypes == object or col in extra_one_hot)):
            print 'Encodying: ',col
            dfall_dummy = pd.get_dummies(dfall[col], prefix='is'+col)
            dfall = dfall.drop([col], axis=1)
            dfall = pd.concat((dfall, dfall_dummy), axis=1)
        elif len(np.unique(dfall[col])) >2:
            print 'Normalizing: ',col, '-',len(np.unique(dfall[col])), 'different values'
            dfall[col] = dfall[[col]].apply(lambda x: (x - x.mean()) / (x.std())) #do not remove the double [] from theright side of the equasion - doesnt work
        else:
            print 'Not touching: ',col

train = dfall[:trainlen]
test = dfall[trainlen:]

test = test.drop(['country_destination'],axis=1)

assert(len_train_before == len(train))
assert(len_test_before == len(test))

Encodying:  gender
Encodying:  signup_method
Encodying:  signup_flow
Encodying:  language
Encodying:  affiliate_channel
Encodying:  affiliate_provider
Encodying:  first_affiliate_tracked
Encodying:  signup_app
Encodying:  first_device_type
Encodying:  first_browser
Encodying:  num_AT_booking_request
Encodying:  num_AD_pending
Normalizing:  num_actions - 1065 different values
Not touching:  start_from_zero
Encodying:  device_from_sessions
Not touching:  bad_devicetype_captured
Normalizing:  account_notification_settings_notifications - 25 different values
Normalizing:  account_payment_methods_index - 13 different values
Normalizing:  account_payout_preferences_payout_preferences - 18 different values
Normalizing:  account_privacy_settings_privacy - 14 different values
Normalizing:  account_transaction_history_transaction_history - 10 different values
Normalizing:  admin_templates_update_cached - 27 different values
Normalizing:  airbnb_picks_wishlists_airbnb_picks - 8 different values
N

In [15]:
#After encoding, drop all columns that have unique values outise NDF because they wont add prediction to non-NDF classes
#cannot drop on dfall because they were created on the one-hot-encoding
print 'dropping cols with unique values'
to_be_dropped = []
for col in train.columns:
    if len(np.unique(train[train['country_destination'] != 'NDF'][col])) ==1:
    #if len(np.unique(train[col])) == 1:
        to_be_dropped.append(col)
        
train = train.drop(to_be_dropped,axis=1)
test = test.drop(to_be_dropped,axis=1)
print 'Dropped', len(to_be_dropped), 'columns: ',to_be_dropped

dropping cols with unique values
Dropped 114 columns:  ['account_payment_methods_index', 'cancellation_policy_click', 'click_about_host_click', 'click_amenities_click', 'click_reviews_click', 'create_payment_instrument_create', 'deactivate_user_account_delete', 'delete_listing_description_delete', 'delete_payment_instrument_destroy', 'empty_braintree_client_token', 'empty_disaster_action', 'empty_phone_verification', 'empty_satisfy', 'empty_track_activity', 'host_refund_guest_refund_guest_cancellation', 'host_respond_page_accept_decline', 'host_standard_suspension_deactivated', 'message_post_special_offer', 'modify_reservations_patch', 'modify_users_patch', 'move_map_click', 'p3_view', 'p4_book', 'phone_numbers_delete', 'photos_click', 'profile_reviews_reviews', 'reservations_delete', 'set_default_payment_instrument_set_default', 'share_click', 'tos_2014_tos_2014', '-unknown-_acculynk_bin_check_failed', '-unknown-_acculynk_pin_pad_error', '-unknown-_acculynk_pin_pad_success', '-unknown

In [16]:
######################       SELECT K BEST FEARURES METHOD
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2, f_classif

def SelectKBestFeatures(train, target, k, debug=False):
    k_best = SelectKBest(f_classif, k='all')# score all features    
    k_best.fit(train, target) 
    scores = k_best.scores_     
    unsorted_pairs = zip(train.columns, scores)# joins the labels with the values 
    sorted_pairs = list(reversed(sorted(unsorted_pairs, key=lambda x: x[1])))

    k_best_features = dict(sorted_pairs[:k])
    if debug:
        print sorted_pairs
        print "{0} best features: {1}\n".format(k, k_best_features.keys())

    return k_best_features
# import operator
# k_best_features
# sorted_x  = sorted(k_best_features.items(), key=operator.itemgetter(1))
# (sorted_x).reverse()
# print sorted_x

#CREATE LIST OF COMBINATIONS BETWEEN FEATURES
#new_list = k_best_features.keys()
#import itertools
#all = []
#for i in range(len(new_list)):
#    if i !=0: #already looked at individual features on step 1. Start by 2X2
#        all.extend([sorted(l) for l in itertools.combinations(new_list, i+1)])

In [17]:
######################SELECT K-BEST FEATURES - filtering out NDFs
#Remove columns that we dont have data on the training side, otherweise it will messup the select best features
#If I want to automate the removal, it has to be here; it cannot be before the df_all split; they wont be removed because there
#are values on the test rows but we cant do anything with them because there is nothing on train

import operator
#k_best_features = SelectKBestFeatures(train[train['country_destination'] != 'NDF'].ix[:, train.columns != 'country_destination'],target[target != 'NDF'], 1000, False)
k_best_features = SelectKBestFeatures(train.ix[:, train.columns != 'country_destination'],target, 100, False)
sorted_x  = sorted(k_best_features.items(), key=operator.itemgetter(1))
(sorted_x).reverse()

#print sorted_x
for a,b in sorted_x:
     print a,b

isnum_AT_booking_request_0 1077.4059948
isnum_AT_booking_request_1 991.679578309
isage_bucket_-1 989.443022457
PopInKtoUS 984.831965543
PopInKtoGB 984.535632153
PopInKtoCA 983.605339114
PopInKtoAU 981.424585907
PopInKtoFR 978.684418889
PopInKtoNL 952.182235412
PopInKtoDE 943.750706183
PopInKtoPT 942.425798601
PopInKtoIT 908.720259442
PopInKtoES 899.7677045
isnum_AD_pending_0 857.583626757
pending_pending 793.863637725
isnum_AD_pending_1 792.292685375
p5_requested 624.672744437
-unknown-_verify 519.830254663
post_checkout_action_requested 430.737002059
isgender_-unknown- 422.063096018
create_phone_numbers_create 313.274952751
isreceived_confirm_email_0.0 279.759900369
isreceived_confirm_email_1.0 279.759900369
your_trips_travel_plans_current 229.740820023
at_checkpoint_at_checkpoint 175.623947073
message_post_empty 172.982465528
secs_elapsed 160.280057515
isgender_FEMALE 153.716400511
isage_bucket_25-29 147.617298091
issignup_app_Web 147.118309256
isfirst_browser_-unknown- 145.389501685

In [40]:
#SCORE AND CROSS VALIDATE
#This methos applies cross validation on the train set. It is configured to always do on two ways refered as model1 and model2
#mode1 (M1) and model2 (M2F) dont actually refer to the model itself but he way the model is being evaluated. 
#model 2 itself is also computed in 2 ways, full dataset (M2F) and noNDF (just M2)
#on model1: the prediction is in one categorical column; on model2 each possible prediction value corresponds to a dummy colum
#so it fits N model on the same dataset, N being the number of possible categories

from sklearn.cross_validation import StratifiedShuffleSplit
def crossValidate(model1, model2, xgbparams, trainCV, targetCV, nfolds, debug, runM1=False, runM2=False, runM2F=False, runM3=False):
    print "Cross Validating..."
    cv = StratifiedShuffleSplit(targetCV['country_destination'], nfolds, test_size  = 0.3, random_state = 42)

    foldercount=0
    list_scores1 = [] 
    list_scores2 = [] 
    list_scores2F= [] 
    list_scores3= [] 
    avgscore1 = 0
    avgscore2 = 0
    avgscore2F = 0    
    avgscore3 = 0  
    
    for train_index, test_index in cv: 
        if debug:
            print 'Go for fold: ',foldercount    
            #print("TRAIN:", train_index, "TEST:", test_index)  
        train_features = []
        train_labels = []
        test_features = []
        test_labels = []    
                
        train_features, train_labels = trainCV.iloc[train_index], targetCV.iloc[train_index]
        test_features, test_labels =   trainCV.iloc[test_index],  targetCV.iloc[test_index]        
          
        
        #create an array from a <class 'pandas.core.series.Series'> to attch to the final DF - mainly for debugging
        truth = []
        for i in test_labels['country_destination']:
            truth.append(i)
        
        #these are for the model being trained without NDF, so I have to filter the NDF values and the NDF column
        train_featuresNoNDF = train_features[train_features['country_destination'] != 'NDF'].ix[:, train_features.columns != 'country_destination']
        test_featuresNoNDF = test_features.ix[:, test_features.columns != 'country_destination']
               
        #this is for M1 and M2F  so i just remove the NDF column that was passed in for the previous filter
        train_features = train_features.ix[:, train_features.columns != 'country_destination']
        test_features = test_features.ix[:, test_features.columns != 'country_destination']

        
######## MODEL1 #####################################
        if runM1:
            model1.fit(train_featuresNoNDF, train_labels[train_labels['country_destination'] != 'NDF']['country_destination']) #train_labels['country_destination']
            pred_array = model1.predict_proba(test_featuresNoNDF)     


            my_debugger = []
            list_of_labels1 = []        
            for idx, val in enumerate(pred_array):
                mypred = (zip(model1.classes_, pred_array[idx]))            
                sorted_by_pred = sorted(mypred, key=lambda tup: tup[1], reverse=True)
                sorted_by_pred = sorted_by_pred[:5]                      
                only_labels =  [x[0] for x in sorted_by_pred]            
                #apply prob correction?

                list_of_labels1.append(only_labels)
                if debug:
                    my_debugger.append ([x[0] +': '+ str(x[1]) for x in sorted_by_pred])        


            #list_of_labels contains a list where each element is a list with the top5 predictions for that ID, for example:
            #[['NDF', 'US', 'other', 'ES', 'NL'], ['NDF', 'US', 'other', 'FR', 'IT'],]         
            preds_df = pd.DataFrame(list_of_labels1, columns=('pred1', 'pred2', 'pred3', 'pred4', 'pred5'))        
            scores1 = score_predictions(preds_df, truth)        
            avgscore1 = np.average(scores1)
            list_scores1.append(avgscore1)       

        
        #This is used by M2 and M2F:
        #get the ids from the original train ids based on the indexes we are testing:
        ids = []
        for i in train_ids.iloc[test_index]: # train_ids
            ids.append(i)    
            
            
######## MODEL2 #####################################
        if runM2:
            #pred_list contains the country and a list of probabilities the row belong to that country for that country
            #1. Do the prediction for each country
            pred_list = []
            for col in targetCV.columns:
                if col != 'country_destination' and col != 'NDF': #we dont want to predict NDF on this               
                    model2.fit(train_featuresNoNDF, train_labels[train_labels['country_destination'] != 'NDF'][col])
                    pred = model2.predict_proba(test_featuresNoNDF) 
                    pred_list.append([col, pred])


            #2.Create the result data frame
            dfResult = pd.DataFrame()               
            dfResult['id'] = ids


            for a in pred_list: # gets the predictions, one country at a time
                dest = a[0] #get country name"
                predictions = a[1] #Get the "True" prediction. logreg.classes_ should return #array([False,  True], dtype=bool)  
                values = []
                for p in predictions:
                    values.append(p[1])
                dfResult[dest] = values                        
            #at this point, dfResult contains one row per ID with one colum for each country and 
            #the True prediction probability for that country. 

            #The following Lines have been deprecated and subistituded by the get_top5_on_lis_of_labels3 function
            #Unpivot: #result = unpivot_from_resultdf_to_dict(dfResult)
            #Get topp5: #list_of_labels = pivot_back_with_topn(result,5, False, dfResult)        
            #return dfResult
            #function "2" applies the "NDF correction" because NDF wasn't one of the predictions
            #return dfResult
            list_of_labels2 = get_top5_on_lis_of_labels2(dfResult, False)

            preds_df = pd.DataFrame(list_of_labels2, columns=('pred1', 'pred2', 'pred3', 'pred4', 'pred5'))        
            scores2 = score_predictions(preds_df, truth)
            avgscore2 = np.average(scores2)
            list_scores2.append(avgscore2)  

######## MODEL2F #####################################
#pretty much the same as the previous model - except fot the function being called 
#TODO: maybe unify the code in one method
        if runM2F:
            pred_listF = []
            for col in targetCV.columns:
                if col != 'country_destination': 
                    if col == 'US':
                        model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                                           gamma=2, learning_rate=0.3, max_delta_step=2, max_depth=5,
                                           min_child_weight=1, missing=None, n_estimators=30, nthread=-1,
                                           objective='binary:logistic', reg_alpha=0, reg_lambda=1,
                                           scale_pos_weight=1, seed=0, silent=True, subsample=1)
                    elif col == 'NDF':
                        model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                                           gamma=1, learning_rate=0.3, max_delta_step=2, max_depth=4,
                                           min_child_weight=1, missing=None, n_estimators=40, nthread=-1,
                                           objective='binary:logistic', reg_alpha=0, reg_lambda=1,
                                           scale_pos_weight=1, seed=0, silent=True, subsample=1)
                    else:
                        model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                                           gamma=1, learning_rate=0.3, max_delta_step=2, max_depth=4,
                                           min_child_weight=1, missing=None, n_estimators=40, nthread=-1,
                                           objective='binary:logistic', reg_alpha=0, reg_lambda=1,
                                           scale_pos_weight=1, seed=0, silent=True, subsample=1)
                
                    model2.fit(train_features, train_labels[col])
                    pred = model2.predict_proba(test_features) 
                    pred_listF.append([col, pred])


            #2.Create the result data frame
            dfResult2F = pd.DataFrame()                
            dfResult2F['id'] = ids# can use the same ids created on the previous step

            for a in pred_listF: # gets the predictions, one country at a time
                dest = a[0] #get country name"
                predictions = a[1] #Get the "True" prediction. logreg.classes_ should return #array([False,  True], dtype=bool)  
                values = []
                for p in predictions:
                    values.append(p[1])
                dfResult2F[dest] = values                
        
            list_of_labels2F = get_top5_on_lis_of_labels2F(dfResult2F, False)   


            preds_df2F = pd.DataFrame(list_of_labels2F, columns=('pred1', 'pred2', 'pred3', 'pred4', 'pred5'))        
            scores2F = score_predictions(preds_df2F, truth)
            avgscore2F = np.average(scores2F)
            list_scores2F.append(avgscore2F)          

######## MODEL3 #####################################
        if runM3:
            pred_listM3 = []
            for col in targetCV.columns:
                if col != 'country_destination':
                    xgbmodel = xgb.train(params=xgbparams, dtrain=xgb.DMatrix(train_features.values, train_labels[col].values), num_boost_round=100)                    
                    pred = xgbmodel.predict(xgb.DMatrix(test_features.values)) 
                    pred_listM3.append([col, pred])

            #2.Create the result data frame
            dfResult3 = pd.DataFrame()               
            dfResult3['id'] = ids


            for a in pred_listM3: # gets the predictions, one country at a time
                dest = a[0] #get country name"
                predictions = a[1] #Get the "True" prediction. logreg.classes_ should return #array([False,  True], dtype=bool)  
                values = []
                for p in predictions:
                    values.append(p[1])
                dfResult3[dest] = values     
                
            list_of_labels3 = get_top5_on_lis_of_labels2F(dfResult3, False)

            preds_df3 = pd.DataFrame(list_of_labels3, columns=('pred1', 'pred2', 'pred3', 'pred4', 'pred5'))        
            scores3 = score_predictions(preds_df3, truth)
            avgscore3 = np.average(scores3)
            list_scores3.append(avgscore3)              

        
##################
        if debug:
            print 'fold:', foldercount, 'score:', avgscore1,'|',avgscore2,'|',avgscore2F, '|',avgscore3
            ################MODEL1 DEBUG
            if runM1:
                debbuger_df = pd.DataFrame(my_debugger, columns=('pred1', 'pred2', 'pred3', 'pred4', 'pred5'))   
                debbuger_df['t']  = truth            
                debbuger_df['score']  = scores1
                debbuger_df['id'] = ids        
                debbuger_df.to_csv('C:\\git\\Airbnb\\LOG_EvalModel1__'+str(foldercount)+'.csv',index=False)   

            ################MODEL2 DEBUG
            if runM2:
                dfResult['t']  = truth            
                dfResult['score']  = scores2
                foo = pd.concat((dfResult, preds_df), axis=1, ignore_index=False)
                foo.to_csv('C:\\git\\Airbnb\\LOG_EvalModel2__'+str(foldercount)+'.csv',index=False)
            
            ################MODEL2F DEBUG
            if runM2F:
                dfResult2F['t']  = truth            
                dfResult2F['score']  = scores2F
                foo = pd.concat((dfResult2F, preds_df2F), axis=1, ignore_index=False)
                foo.to_csv('C:\\git\\Airbnb\\LOG_EvalModel2F__'+str(foldercount)+'.csv',index=False)

            ################MODEL3 DEBUG
            if runM3:
                dfResult3['t']  = truth            
                dfResult3['score']  = scores3
                foo = pd.concat((dfResult3, preds_df3), axis=1, ignore_index=False)
                foo.to_csv('C:\\git\\Airbnb\\LOG_EvalModel3__'+str(foldercount)+'.csv',index=False)                
            
        foldercount +=1

    return np.average(list_scores1), np.average(list_scores2), np.average(list_scores2F), np.average(list_scores3)

# model1 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
#        gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
#        min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
#        objective='multi:softprob', reg_alpha=0, reg_lambda=1,
#        scale_pos_weight=1, seed=0, silent=True, subsample=1)

# model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
#        gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
#        min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
#        objective='binary:logistic', reg_alpha=0, reg_lambda=1,       
#        scale_pos_weight=1, seed=0, silent=True, subsample=1)

# xgbparams = {'eta': 0.2,
#       'max_depth': 6,
#       'subsample': 0.5,
#       'colsample_bytree': 0.5,
#       'objective': 'binary:logistic',
#       'num_class': 2}  

# #features =  train.ix[:, train.columns != 'country_destination'].columns  #k_best_features.keys(
# #features = train.columns
# #train = train[:30000]
# #dfpivottarget = dfpivottarget[:30000]

# mycolumns = ['feature_list', 'scoreM1', 'ScoreM2NoNDF', 'ScoreM2Full', 'ScoreM3']
# resultdf2 = pd.DataFrame(columns=mycolumns)

# features = train [ [col for col in train.columns if col not in ['start_from_zero', 
#                                                                 'isstart_from_zero_0', 'isstart_from_zero_1',
#                                                                 'num_actions', ]] ].columns
 
# score1, ScoreM2NoNDF, ScoreM2Full, Score3 = crossValidate(model1, model2, xgbparams, train[features], dfpivottarget, 
#                                                           nfolds = 1, 
#                                                           debug= True,
#                                                           runM1=False, 
#                                                           runM2=False, 
#                                                           runM2F=True, 
#                                                           runM3=False)

# resultdf2.loc[len(resultdf2)] = (features, score1, ScoreM2NoNDF, ScoreM2Full, Score3)
# resultdf2

In [19]:
#model = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
#            metric_params=None, n_neighbors=4, p=2, weights='uniform')

# model = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
#               max_depth=7, max_features=9, max_leaf_nodes=None,
#               min_samples_leaf=1, min_samples_split=2,
#               min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
#               oob_score=False, random_state=None, verbose=0,
#               warm_start=False)
#model = RandomForestClassifier(n_estimators=50,  criterion='gini')
#Best
#model = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
#                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0) 

#model = XGBClassifier(max_depth=6, learning_rate=0.25, n_estimators=30,
#                    objective='multi:softprob', subsample=0.7, colsample_bytree=0.6, seed=0)     

model1 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)     

xgbparams = {'eta': 0.1,
          'max_depth': 6,
          'subsample': 1,
          'colsample_bytree': 0.9,
          'objective': 'binary:logistic',
          'num_class': 2} 

In [41]:
#RUN CROSS VALIDATION
mycolumns = ['feature_list', 'scoreM1', 'ScoreM2NoNDF', 'ScoreM2Full', 'ScoreM3']
resultdf2 = pd.DataFrame(columns=mycolumns)


#features = train.columns# train.ix[:, train.columns != 'secs_elapsed'].columns #  #k_best_features.keys()
features = train [ [col for col in train.columns if col not in ['num_actions'] ]].columns
#features = train.columns
#features = k_best_features.keys()
#features.append('country_destination')


    
score1, ScoreM2NoNDF, ScoreM2Full, Score3 = crossValidate(model1, model2, xgbparams, train[features], dfpivottarget, 
                                                          nfolds = 3, 
                                                          debug= True,
                                                          runM1=False, 
                                                          runM2=False, 
                                                          runM2F=True, 
                                                          runM3=False)

resultdf2.loc[len(resultdf2)] = (features, score1, ScoreM2NoNDF, ScoreM2Full, Score3)
resultdf2

Cross Validating...
Go for fold:  0
fold: 0 score: 0 | 0 | 0.855782496037 | 0
Go for fold:  1
fold: 1 score: 0 | 0 | 0.85418516049 | 0
Go for fold:  2
fold: 2 score: 0 | 0 | 0.855808203666 | 0


feature_list  scoreM1  ScoreM2NoNDF  \
0  Index([u'start_from_zero', u'bad_devicetype_ca...      NaN           NaN   

   ScoreM2Full  ScoreM3  
0     0.855259      NaN

In [26]:
XGBClassifier()

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [27]:
model2

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [37]:
from sklearn.grid_search import GridSearchCV
def suggestXGBClassifierModel2(train,target):
    xgb_params = [{                   
                   'gamma' : range(1,3)
                 , 'learning_rate' : [0.3, 0.4, 0.5]
                 , 'max_delta_step': range (1,3)
                 , 'max_depth': range(3,6)
                 , 'n_estimators' : [30,40]
                }]

    clf = GridSearchCV(XGBClassifier(objective='binary:logistic'), xgb_params) # , scoring='f1'
    
    c = ['FR','NDF','US','GB','IT','NL','other','CA','DE','ES','PT', 'AU']
    for col in c:
        if col != 'country_destination': 
            print 'Testing', col
            best = clf.fit(train, target[col])
            
            f = open('C:\\git\\Airbnb\\GridSearch\\M2F'+col+'.txt', 'w')
            f.write(col)
            f.write('\n')
            f.write(str(best.best_estimator_))
            f.close()
            
print "Start" + time.strftime("%c")    
suggestXGBClassifierModel2( train.ix[:, train.columns != 'country_destination'], dfpivottarget)
print "End" + time.strftime("%c")       

Start02/11/16 09:54:03
Testing FR
Testing NDF
Testing US
Testing GB


KeyboardInterrupt: 

In [35]:
best.best_estimator_

NameError: name 'best' is not defined

In [70]:
#START ACTUAL PREDCTION

In [21]:
# model = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
#              max_depth=7, max_features=9, max_leaf_nodes=None,
#              min_samples_leaf=1, min_samples_split=2,
#              min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
#              oob_score=False, random_state=None, verbose=0,
#              warm_start=False)

# model = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
#             metric_params=None, n_neighbors=4, p=2, weights='uniform')

#model = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
#                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0) 

#model = XGBClassifier(max_depth=6, learning_rate=0.25, n_estimators=43,
#                    objective='multi:softprob', subsample=0.6, colsample_bytree=0.6, seed=0) 

model1 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
       objective='multi:softprob', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.5,
       gamma=1, learning_rate=0.25, max_delta_step=2, max_depth=4,
       min_child_weight=0, missing=None, n_estimators=40, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)    

In [71]:
############################### EVAL  MODEL1
#features = train.columns#  k_best_features.keys() #


model1.fit(train[features], target)
pred_array = model1.predict_proba(test[features])


list_of_labels1 = []
print "Start" + time.strftime("%c")
for idx, val in enumerate(pred_array):
    mypred = (zip(model1.classes_, pred_array[idx]))
    sorted_by_pred = sorted(mypred, key=lambda tup: tup[1], reverse=True)
    sorted_by_pred = sorted_by_pred[:5]
    only_labels =  [x[0] for x in sorted_by_pred]
    
    #unpivot for submission:
    for country in only_labels:   
        list_of_labels1.append([test_ids[idx], country] )
print "End1 " + time.strftime("%c")        
        
preds_df = pd.DataFrame(list_of_labels1, columns=('id', 'country'))       
print "End2 " + time.strftime("%c")        

Start01/07/16 18:32:53
End1 01/07/16 18:32:56
End2 01/07/16 18:32:56


In [72]:
######EXPORT MODEL 1
preds_df.to_csv('C:\\git\\Airbnb\\m1.csv',index=False) 

In [42]:
#######MODEL2
#features = train.columns#  k_best_features.keys() #
#features = train.ix[:, train.columns != 'country_destination'].columns  #k_best_features.keys()
features = train [ [col for col in train.columns if col not in ['num_actions', 'country_destination'] ]].columns

Model2_list = []

for col in dfpivottarget.columns:
    if col != 'country_destination':
        print col
        if col == 'US':
            model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                               gamma=2, learning_rate=0.3, max_delta_step=2, max_depth=5,
                               min_child_weight=1, missing=None, n_estimators=30, nthread=-1,
                               objective='binary:logistic', reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=1, seed=0, silent=True, subsample=1)
        elif col == 'NDF':
            model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                               gamma=1, learning_rate=0.3, max_delta_step=2, max_depth=4,
                               min_child_weight=1, missing=None, n_estimators=40, nthread=-1,
                               objective='binary:logistic', reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=1, seed=0, silent=True, subsample=1)
        else:
            model2 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
                               gamma=1, learning_rate=0.3, max_delta_step=2, max_depth=4,
                               min_child_weight=1, missing=None, n_estimators=40, nthread=-1,
                               objective='binary:logistic', reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=1, seed=0, silent=True, subsample=1)        
        model2.fit(train[features], dfpivottarget[col])
        pred = model2.predict_proba(test[features])  

        Model2_list.append([col, pred])

AU
CA
DE
ES
FR
GB
IT
NDF
NL
other
PT
US


In [43]:
#Model2_list.classes_ # check it is array([False,  True], dtype=bool)
dfResult = pd.DataFrame()
dfResult['id'] = test_ids

for a in Model2_list:
    dest = a[0] 
    predictions = a[1]
    values = []
    for p in predictions:
        values.append(p[1])
    dfResult[dest] = values

In [44]:
list_of_labels2 = get_top5_on_lis_of_labels2F(dfResult, True)

In [45]:
preds_df = pd.DataFrame(list_of_labels2, columns=('id', 'country'))
preds_df.to_csv('C:\\git\\Airbnb\\official_multymodels.csv',index=False) 

In [29]:
dfResult

id        AU        CA        DE        ES        FR        GB  \
0      0010k6l0om  0.000721  0.003765  0.001562  0.004314  0.009361  0.003403   
1      0031awlkjq  0.000732  0.005212  0.002583  0.019130  0.020271  0.013235   
2      00378ocvlh  0.001272  0.021267  0.006654  0.009221  0.045021  0.035492   
3      0048rkdgb1  0.000822  0.001132  0.000606  0.002092  0.003339  0.002506   
4      0057snrdpu  0.002558  0.002927  0.002802  0.006492  0.013856  0.003712   
5      005v5uf4dh  0.000822  0.001132  0.000606  0.002092  0.003880  0.003505   
6      0063bawn05  0.001502  0.003757  0.003878  0.004974  0.013084  0.004728   
7      006ml14zc1  0.000948  0.009278  0.001625  0.065290  0.045447  0.016711   
8      0075z9e9xv  0.000941  0.004456  0.005314  0.021898  0.032308  0.008743   
9      00an0o6c07  0.000795  0.001494  0.001814  0.005305  0.014460  0.006022   
10     00bqkh0g5i  0.000695  0.001488  0.000730  0.002497  0.006193  0.003648   
11     00c8w2zol1  0.000885  0.003446  0.001135  0.004939  0.010024  0.004004   
12     00cdlcsaxu  0.001222  0.004201  0.003294  0.007005  0.009211  0.004189   
13     00cu8ilsh0  0.000743  0.004725  0.003041  0.007246  0.019132  0.010462   
14     00d41chbjd  0.002961  0.008350  0.003783  0.006093  0.017836  0.007806   
15     00dj8as8yl  0.001727  0.012039  0.001776  0.011792  0.050998  0.025912   
16     00du2h6jcx  0.000822  0.001132  0.000606  0.002276  0.004290  0.002350   
17     00ep2pem90  0.000799  0.002491  0.002148  0.006649  0.013203  0.009047   
18     00epe0uaxo  0.000816  0.001745  0.001969  0.008224  0.016930  0.010981   
19     00eqsrxf1a  0.000627  0.001115  0.000704  0.001227  0.006960  0.004677   
20     00gpn5xx7t  0.002371  0.002068  0.000917  0.002840  0.005279  0.004394   
21     00ipxpr25h  0.001250  0.010475  0.004778  0.009903  0.030351  0.018339   
22     00j9fxfvut  0.003099  0.002302  0.001590  0.005837  0.009100  0.004453   
23     00jcwpqbl2  0.001602  0.003275  0.001517  0.003437  0.012333  0.006683   
24     00jp2043rl  0.000527  0.002092  0.001291  0.004337  0.008717  0.004292   
25     00kbbr6qom  0.002822  0.004611  0.002016  0.006994  0.016288  0.006350   
26     00kpv7ok66  0.004434  0.010302  0.006662  0.004313  0.035933  0.018348   
27     00lglrzikw  0.000956  0.002836  0.002124  0.008903  0.015212  0.011123   
28     00rc560l5d  0.001570  0.001065  0.001170  0.001481  0.004976  0.002761   
29     00rhdi533j  0.003057  0.009016  0.007011  0.025622  0.019679  0.006455   
...           ...       ...       ...       ...       ...       ...       ...   
62066  zzcb9m931d  0.001387  0.003997  0.001270  0.003472  0.008174  0.005046   
62067  zzcmc8ro2p  0.002664  0.007561  0.002510  0.008614  0.032889  0.023841   
62068  zzdkfqh4rc  0.000972  0.002386  0.002304  0.006026  0.009237  0.006273   
62069  zzedhgimoi  0.001038  0.001135  0.000963  0.003252  0.005726  0.003523   
62070  zzessy9eef  0.001261  0.002314  0.001954  0.003775  0.011069  0.007488   
62071  zzezu577ji  0.000563  0.001132  0.000606  0.002092  0.003339  0.002555   
62072  zzfr7654fu  0.001056  0.006875  0.001752  0.007191  0.014763  0.009647   
62073  zzi55292q1  0.001118  0.009054  0.010727  0.009980  0.032432  0.024354   
62074  zzj58fjo9k  0.002495  0.015042  0.002691  0.007534  0.054843  0.012633   
62075  zzjoud57mw  0.001050  0.001494  0.002136  0.006947  0.013652  0.003797   
62076  zzjqad3rab  0.000286  0.001131  0.000679  0.002879  0.005856  0.003569   
62077  zzjsos7yaa  0.001245  0.010963  0.002384  0.006929  0.043920  0.016059   
62078  zzknfsp59t  0.002935  0.005152  0.002572  0.007217  0.012060  0.011365   
62079  zzknio9yie  0.000738  0.001126  0.000539  0.002939  0.002707  0.001707   
62080  zzl7v39uqc  0.000495  0.003399  0.003194  0.006199  0.019994  0.008412   
62081  zzn7wvythd  0.005460  0.012016  0.001175  0.008694  0.025392  0.016169   
62082  zznbshw5p7  0.000961  0.001798  0.000957  0.002885  0.004562  0.003048   
62083  zzngxvqfm5  0.000450  0.002135  0.00123

In [43]:
foo =dfResult[dfResult['id']=='j5kxtvyc9b']
foo

id       AU        CA        DE        ES        FR        GB  \
0  j5kxtvyc9b  0.00064  0.000915  0.001786  0.007188  0.018538  0.009074   

         IT        NL     other        PT        US  
0  0.013322  0.001011  0.022586  0.001541  0.077482

In [50]:
get_top5_on_lis_of_labels2(foo, True)

[['j5kxtvyc9b', 'US'],
 ['j5kxtvyc9b', 'other'],
 ['j5kxtvyc9b', 'FR'],
 ['j5kxtvyc9b', 'IT'],
 ['j5kxtvyc9b', 'GB']]

In [61]:
it(1,2)('ABCDEFG')

('B', 'C')

In [97]:
string = 'aabbcdd'

In [98]:
len(string)

7